In [2]:
import pandas as pd 
import re

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

from sklearn.metrics import classification_report, plot_confusion_matrix

from bs4 import BeautifulSoup # if installed

In [3]:
import nltk

In [4]:
path = ''

In [5]:
df = pd.read_csv(f"{path}bdd_avis_restaurant.csv",sep="|")

# Tri des données non françaises

In [6]:
keywords = ["and","not","also","maybe","why","where","time",
            "work","week","year","way","good","new","first","last","great","little",
           "own","other","old","right","high","big","small","early","was",
            "lunch","dinner","big","great","delicious","disgusting","eat"]

In [7]:
numero_drop = []

for i in range(len(df["avis"])):
    liste = df["avis"][i].split()
    anglais = 0
    j=0
    
    for j in keywords : 
        if any(j==s for s in liste):
            anglais = 1
    if anglais ==1:
        numero_drop.append(i)
    

In [8]:
df = df.drop(numero_drop)

In [9]:
df = df[["nom","prix","etoile","avis"]]
df = df.sample(frac=0.8, random_state = 25)
df = df.drop(df_train.index)

# Présentation des données

In [12]:
df_train.describe()

,nom,prix,etoile,avis
count,3166,3166,3166,3166
unique,254,5,6,2872
top,L'Ouest,2,4,En savoir plus
freq,91,1896,1159,141


In [10]:
df_train['etoile'].value_counts()

4         1159
5          947
3          632
2          292
1          135
etoile       1
Name: etoile, dtype: int64

In [21]:
X_train = df_train['avis']
y_train = df_train['etoile']

In [22]:
vectorizer = TfidfVectorizer()

vectorizer.fit(X_train)

TfidfVectorizer()

In [23]:
vectorizer.get_feature_names_out()

array(['00', '00h30', '01', ..., 'îlot', 'ôter', '王老吉'], dtype=object)

In [24]:
X_test = df_test['avis']
y_test = df_test['etoile']

In [25]:
x_tmp = vectorizer.transform(df_train)
x_tmp.shape

(4, 15628)

In [26]:
tfidf = TfidfVectorizer()

tfidf.fit(X_train)
X_tfidf = tfidf.transform(X_train)
X_tfidf.shape
x_test_idf = tfidf.transform(X_test)

In [27]:
svc = LinearSVC(tol=1e-6,loss="squared_hinge",fit_intercept=True,multi_class="ovr",class_weight="balanced",max_iter=10000)

svc.fit(X_tfidf, y_train)

LinearSVC(class_weight='balanced', max_iter=10000, tol=1e-06)

In [28]:
xtest_tmp = tfidf.transform(X_test)
xtest_tmp.shape

(792, 15628)

In [29]:
y_predtest = svc.predict(x_test_idf)

In [30]:
y_predtest2 = svc.predict(xtest_tmp)

In [31]:
print(classification_report(y_test, y_predtest, digits=4 ))

              precision    recall  f1-score   support

           1     0.4400    0.2619    0.3284        42
           2     0.3824    0.3662    0.3741        71
           3     0.4241    0.4379    0.4309       153
           4     0.4933    0.5193    0.5060       285
           5     0.5690    0.5690    0.5690       239
      etoile     1.0000    1.0000    1.0000         2

    accuracy                         0.4924       792
   macro avg     0.5515    0.5257    0.5347       792
weighted avg     0.4913    0.4924    0.4905       792

